In [1]:
import os
#hfhome_dir="/data/cmpe249-fa23/Huggingfacecache"
hfhome_dir=os.path.join('D:',os.sep, 'Cache','huggingface')
#os.environ['TRANSFORMERS_CACHE'] = hfhome_dir
os.environ['HF_HOME'] = hfhome_dir
#os.environ['HF_HUB_CACHE'] = os.path.join(hfhome_dir, 'hub')
#os.environ['HF_DATASETS_CACHE'] = hfhome_dir
#HF_HUB_OFFLINE=1

In [2]:
from datasets import load_dataset, load_metric
dataset_split = load_dataset(
        "librispeech_asr", #'librispeech_asr'
        'clean',
        split='train.100',
    )

c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset_split

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 28539
})

In [4]:
from datasets import DatasetDict
raw_datasets = DatasetDict()
raw_datasets["train"] = dataset_split
validation_split_percentage = 1
num_validation_samples = raw_datasets["train"].num_rows * validation_split_percentage // 100

raw_datasets["validation"] = raw_datasets["train"].select(range(num_validation_samples))
raw_datasets["train"] = raw_datasets["train"].select(range(num_validation_samples, raw_datasets["train"].num_rows))

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 28254
    })
    validation: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 285
    })
})

In [6]:
raw_datasets = raw_datasets.remove_columns(["speaker_id", "chapter_id", "id"])

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 28254
    })
    validation: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 285
    })
})

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(raw_datasets["train"].remove_columns(["file", 'audio']), num_examples=20)

,text
0,IN THIS HIS OPENING EFFORT TO REGAIN POSSESSION OF CORA DO MY YOUNG MEN LEAVE THE DELAWARES ROOM ON THE MOUNTAINS FOR THEIR HUNTS HE AT LENGTH CONTINUED
1,IS THERE ANYTHING THE LEAST WONDERFUL OR REMARKABLE ABOUT THIS NEIGHBOURHOOD I SHOULD JUST THINK THERE WAS REPLIED THE OTHER MANY THINGS THAT DON'T EXIST ANYWHERE ELSE IN THE WORLD
2,WHEN THEY ARRIVED THERE THEY MARCHED STRAIGHT TO ROME AND AFTER SOME FIGHTING
3,AT THE BOTTOM OF THE STREET HOWEVER SHE LOOKED BACK AGAIN AND THEN NOT AT A WINDOW BUT ISSUING FROM THE DOOR SHE SAW MISS TILNEY HERSELF SHE WAS FOLLOWED BY A GENTLEMAN WHOM CATHERINE BELIEVED TO BE HER FATHER AND THEY TURNED UP TOWARDS EDGAR'S BUILDINGS
4,THEIR MURDERER ALSO LIVED AND TO DESTROY HIM I MUST DRAG OUT MY WEARY EXISTENCE I KNELT ON THE GRASS AND KISSED THE EARTH AND WITH QUIVERING LIPS EXCLAIMED BY THE SACRED EARTH
5,AS THE ART IS REAL SHE TOLD ME HOW POOR DEAR LADY BALDOCK COMMUNICATED TO YOU HER UNHAPPINESS ABOUT HER DAUGHTER IN A MANNER THAT MADE EVEN ME LAUGH AND WOULD MAKE THOUSANDS LAUGH IN DAYS TO COME WERE IT EVER TO BE PUBLISHED
6,SOMETIMES ON SUMMER AFTERNOONS SHE CAME OUT OF THE HOUSE AND GOT INTO HER CARRIAGE DISMISSING THE DRIVER SHE TOOK THE REINS IN HER OWN HANDS AND DROVE OFF AT TOP SPEED THROUGH THE STREETS
7,THE OPENING VERY TIGHT FOR IT IS MANIFEST THAT THE TIE MODERATELY STRAIGHTENED WHILE ADEQUATE TO HINDER THE BLOOD ALREADY IN THE ARM FROM RETURNING TOWARDS THE HEART BY THE VEINS CANNOT ON THAT ACCOUNT PREVENT NEW BLOOD
8,AND THAT I WAS RESERVED FOR VENGEANCE THE LAUGHTER DIED AWAY WHEN A WELL KNOWN AND ABHORRED VOICE APPARENTLY CLOSE TO MY EAR
9,AND HANDSOME SHINING MATS OF THE KINNIKINIC SPRINKLED WITH BRIGHT SCARLET BERRIES FROM A PLACE CALLED HUNT'S AT THE END OF THE WAGON ROAD A TRAIL LEADS THROUGH LUSH DRIPPING WOODS NEVER DRY


In [10]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

In [11]:
raw_datasets = raw_datasets.map(remove_special_characters)

In [12]:
show_random_elements(raw_datasets["train"].remove_columns(["file", "audio"]))

,text
0,what do you mean shall i tell you certainly well then i guess that mister willoughby hunts marianne was surprised and confused yet she could not help smiling at the quiet archness of his manner and after a moment's silence said
1,within that region where the sweet west wind rises to open the new leaves wherewith europe is seen to clothe herself afresh not far off from the beating of the waves behind which in his long career the sun sometimes conceals himself from every man
2,yet they went there regularly of their own accord though they betrayed no greater delight in the experience than ourselves on the whole the existence of these olympians seemed to be entirely void of interests
3,that their curiosity was now on the point of being gratified
4,no replied uncle john the merricks are out of elmhurst now and it returns to its rightful owners you owe me nothing my lad
5,indulging in faint and distant hopes of cherishing that life that i now abhor but while i was in the city uncertain what to do as i could not find don fernando i heard notice given by the public crier offering a great reward to anyone who should find me
6,i was extremely perplexed i will permit myself the liberty of saying i would fain believe
7,any child could see that his eyes were tired and his mouth was sad when he was not speaking i'm doing pretty well thank you said the man with a delightful smile
8,asked sellen colouring up to the roots of his hair i want it and a pair of fire tongs sellen gave him the required articles took his sketching stool and sat down on the pieces of cardboard as if he were guarding a treasure
9,the latter was silent brooding over the confidence which philip had apparently received but which was withheld from him he did not yet know of the culminating point of philip's proposed journey to london


In [14]:
import transformers
from transformers import (
    AdamW,
    SchedulerType,
    Wav2Vec2Config,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForPreTraining,
    get_scheduler,
    is_wandb_available,
    set_seed,
)
from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices

In [13]:
model_name_or_path="patrickvonplaten/wav2vec2-base-v2"

In [15]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)

In [17]:
feature_extractor.sampling_rate

16000

In [18]:
feature_extractor.do_normalize

True

In [16]:
import datasets
# make sure that dataset decodes audio with correct sampling rate
raw_datasets = raw_datasets.cast_column(
    "audio", datasets.features.Audio(sampling_rate=feature_extractor.sampling_rate)
)

In [19]:
# set max & min audio length in number of samples
max_duration_in_seconds = 5 
min_duration_in_seconds = 3   
max_length = int(max_duration_in_seconds * feature_extractor.sampling_rate) #80000
min_length = int(min_duration_in_seconds * feature_extractor.sampling_rate) #48000

In [20]:
def prepare_dataset(batch):
    sample = batch["audio"]

    inputs = feature_extractor(
        sample["array"], sampling_rate=sample["sampling_rate"], max_length=max_length, truncation=True
    )
    batch["input_values"] = inputs.input_values[0]
    batch["input_length"] = len(inputs.input_values[0])

    return batch

In [21]:
cache_file_names = None
vectorized_datasets = raw_datasets.map(
        prepare_dataset,
        num_proc=1,
        remove_columns=raw_datasets["train"].column_names,
        cache_file_names=cache_file_names,
    )

Map: 100%|██████████| 285/285 [00:02<00:00, 135.75 examples/s]


In [22]:
if min_length > 0.0:
    vectorized_datasets = vectorized_datasets.filter(
        lambda x: x > min_length,
        num_proc=1,
        input_columns=["input_length"],
    )

Filter: 100%|██████████| 285/285 [00:00<00:00, 31635.44 examples/s]


In [23]:
vectorized_datasets = vectorized_datasets.remove_columns("input_length")

In [24]:
vectorized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_values'],
        num_rows: 27667
    })
    validation: Dataset({
        features: ['input_values'],
        num_rows: 282
    })
})

In [25]:
# 3. Load model
config = Wav2Vec2Config.from_pretrained(model_name_or_path)

In [26]:
# pretraining is only supported for "newer" stable layer norm architecture
# apply_spec_augment has to be True, mask_feature_prob has to be 0.0
if not config.do_stable_layer_norm or config.feat_extract_norm != "layer":
    raise ValueError(
        "PreTraining is only supported for ``config.do_stable_layer_norm=True`` and"
        " ``config.feat_extract_norm='layer'"
    )

In [27]:
# initialize random model
model = Wav2Vec2ForPreTraining(config)

In [28]:
import math
import os
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Union
import torch

@dataclass
class DataCollatorForWav2Vec2Pretraining:
    """
    Data collator that will dynamically pad the inputs received and prepare masked indices
    for self-supervised pretraining.

    Args:
        model (:class:`~transformers.Wav2Vec2ForPreTraining`):
            The Wav2Vec2 model used for pretraining. The data collator needs to have access
            to config and ``_get_feat_extract_output_lengths`` function for correct padding.
        feature_extractor (:class:`~transformers.Wav2Vec2FeatureExtractor`):
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
        mask_time_prob (:obj:`float`, `optional`, defaults to :obj:`0.65`):
            Percentage (between 0 and 1) of all feature vectors along the time axis which will be masked for the contrastive task.
            Note that overlap between masked sequences may decrease the actual percentage of masked vectors.
            The default value is taken from the original wav2vec 2.0 article (https://arxiv.org/abs/2006.11477),
            and results in about 49 percent of each sequence being masked on average.
        mask_time_length (:obj:`int`, `optional`, defaults to :obj:`10`):
            Length of each vector mask span to mask along the time axis in the contrastive task. The default value
            originates from the original wav2vec 2.0 article and corresponds to the ``M`` variable mentioned there.
    """

    model: Wav2Vec2ForPreTraining
    feature_extractor: Wav2Vec2FeatureExtractor
    padding: Union[bool, str] = "longest"
    pad_to_multiple_of: Optional[int] = None
    mask_time_prob: Optional[float] = 0.65
    mask_time_length: Optional[int] = 10

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # reformat list to dict and set to pytorch format
        batch = self.feature_extractor.pad(
            features,
            padding=self.padding,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        device = batch["input_values"].device
        batch_size = batch["input_values"].shape[0]

        mask_indices_seq_length = self.model._get_feat_extract_output_lengths(batch["input_values"].shape[-1])
        # make sure masked sequence length is a Python scalar
        mask_indices_seq_length = int(mask_indices_seq_length)

        # make sure that no loss is computed on padded inputs
        if batch.get("attention_mask") is not None:
            # compute real output lengths according to convolution formula
            batch["sub_attention_mask"] = self.model._get_feature_vector_attention_mask(
                mask_indices_seq_length, batch["attention_mask"]
            )

        features_shape = (batch_size, mask_indices_seq_length)

        # sample randomly masked indices
        mask_time_indices = _compute_mask_indices(
            features_shape,
            self.mask_time_prob,
            self.mask_time_length,
            attention_mask=batch.get("sub_attention_mask"),
        )

        # sample negative indices
        sampled_negative_indices = _sample_negative_indices(
            features_shape,
            self.model.config.num_negatives,
            mask_time_indices=mask_time_indices,
        )
        batch["mask_time_indices"] = torch.tensor(mask_time_indices, dtype=torch.long, device=device)
        batch["sampled_negative_indices"] = torch.tensor(sampled_negative_indices, dtype=torch.long, device=device)

        return batch


In [29]:
data_collator = DataCollatorForWav2Vec2Pretraining(
        model=model,
        feature_extractor=feature_extractor,
        pad_to_multiple_of=None,
        mask_time_prob=config.mask_time_prob,
        mask_time_length=config.mask_time_length,
    )

In [36]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB 8.6 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.6 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 13.0 MB/s eta 0:00:00


In [37]:
wer_metric = load_metric("wer")

In [39]:
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = feature_extractor.tokenizer.pad_token_id

    pred_str = feature_extractor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = feature_extractor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [40]:
model.freeze_feature_extractor()

c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1629: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [41]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./output/wav2vec2-base-asr-demo",
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [45]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=vectorized_datasets["train"],
    eval_dataset=vectorized_datasets["validation"],
    tokenizer=feature_extractor,
)

In [46]:
trainer.train()

# Custom Training loop

In [30]:
from torch.utils.data.dataloader import DataLoader
batch_size = 8
train_dataloader = DataLoader(
    vectorized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    vectorized_datasets["validation"], collate_fn=data_collator, batch_size=batch_size
)

In [32]:
# Optimizer
learning_rate=5e-5
adam_beta1=0.9
adam_beta2=0.999
adam_epsilon=1e-8
optimizer = AdamW(
    list(model.parameters()),
    lr=learning_rate,
    betas=[adam_beta1, adam_beta2],
    eps=adam_epsilon,
)

c:\Users\lkk68\.conda\envs\mycondapy310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
gradient_accumulation_steps =1
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)

In [34]:
num_update_steps_per_epoch

3459